In [1]:
!pip install pytest

In [2]:
!pip install pytest-cov

In [3]:
import pandas as pd
import numpy as np

In [4]:
df = pd.read_csv('/content/sample_data/dados_corretos2.csv')

# Colunas com datas mal formatadas, tentar converter sem verificar
df['signup_date'] = pd.to_datetime(df['signup_date'])
df['last_purchase_date'] = pd.to_datetime(df['last_purchase_date'])

# Preencher valores ausentes com 0 sem critério
df = df.fillna(0)

# Converter coluna de churn para número (simplesmente assume que é sim/não)
def ch(x):
    if x == 'yes':
        return 1
    else:
        return 0

df['churn'] = df['churn'].apply(ch)

# Criar coluna de fidelidade manualmente
df['fidelidade'] = df['num_orders'] / (df['days_since_last_purchase'] + 1)

# One-hot encoding errado da categoria de produto
categorias = list(set(df['product_category']))
for i in range(len(categorias)):
    df['cat_' + str(i)] = df['product_category'].apply(lambda x: 1 if x == categorias[i] else 0)

# Codificar payment_method com números arbitrários
def metodo(m):
    if m == 'credit_card':
        return 1
    elif m == 'boleto':
        return 2
    elif m == 'pix':
        return 3
    else:
        return 4

df['payment_method'] = df['payment_method'].apply(metodo)

# Criar feature nova que multiplica total gasto pelo número de pedidos
df['score_gasto'] = df['total_spent'] * df['num_orders']

# Salvar sem verificar se diretório existe
df.to_csv('/content/sample_data/dados_final.csv')


In [5]:
%cd /content/drive/MyDrive/ALURA/4812-EngenhariadeSoftware_para_CiênciadeDados/Churn/src/tratamentos
!pip install -e .

/content/drive/MyDrive/ALURA/4812-EngenhariadeSoftware_para_CiênciadeDados/Churn/src/tratamentos
Obtaining file:///content/drive/MyDrive/ALURA/4812-EngenhariadeSoftware_para_Cie%CC%82nciadeDados/Churn/src/tratamentos
  Preparing metadata (setup.py) ... done
  Attempting uninstall: pre_processamento
    Found existing installation: pre_processamento 0.1.0
    Uninstalling pre_processamento-0.1.0:
      Successfully uninstalled pre_processamento-0.1.0
  Running setup.py develop for pre_processamento


In [6]:
# Criar CSV teste
df = pd.DataFrame({
    'customer_id': [1, 2],
    'signup_date': ['2022-01-01', '2022-01-15'],
    'last_purchase_date': ['2022-01-20', '2022-02-01'],
    'product_category': ['eletronico', None],
    'total_spent': [1000.0, 500.0],
    'num_orders': [10, 5],
    'cancel_date': [None, None],
    'data_ingestion': ['2022-02-05', '2022-02-10'],
    'city': ['São Paulo', None],
    'payment_method': ['pix', 'boleto'],
    'product_id': ['A1', 'B2'],
    'churn': ['no', 'yes'],
    'days_since_last_purchase': [10, None]
})

df.to_csv("/content/drive/MyDrive/ALURA/4812-EngenhariadeSoftware_para_CiênciadeDados/Churn/data/clientes_teste.csv", index=False)

In [7]:
from pre_processamento import PreProcessamento

pre_processing = PreProcessamento("/content/drive/MyDrive/ALURA/4812-EngenhariadeSoftware_para_CiênciadeDados/Churn/data/clientes_teste.csv")
pre_processing.executar_pipeline("clientes_processados.csv")

# Ver o resultado
pd.read_csv("clientes_processados.csv")

,customer_id,signup_date,last_purchase_date,total_spent,num_orders,cancel_date,data_ingestion,city,product_id,churn,days_since_last_purchase,fidelidade,score_gasto,cat_eletronico,cat_outro,pay_boleto,pay_pix
0,1,2022-01-01,2022-01-20,1000.0,10,NaN,2022-02-05,São Paulo,A1,0,10.0,0.909091,10000.0,True,False,False,True
1,2,2022-01-15,2022-02-01,500.0,5,NaN,2022-02-10,Desconhecido,B2,1,10.0,0.454545,2500.0,False,True,True,False


In [14]:
!pytest /content/drive/MyDrive/ALURA/4812-EngenhariadeSoftware_para_CiênciadeDados/Churn/tests/

============================= test session starts ==============================
platform linux -- Python 3.11.13, pytest-8.4.1, pluggy-1.6.0
rootdir: /content/drive/MyDrive/ALURA/4812-EngenhariadeSoftware_para_CiênciadeDados/Churn
plugins: cov-6.2.1, anyio-4.9.0, langsmith-0.4.8, typeguard-4.4.4
collected 4 items                                                              

tests/test_core.py ....                                                  [100%]

============================== 4 passed in 0.82s ===============================


In [15]:
%cd /content/drive/MyDrive/ALURA/4812-EngenhariadeSoftware_para_CiênciadeDados/Churn/
!pytest --cov=src/tratamentos/pre_processamento/ tests/

/content/drive/MyDrive/ALURA/4812-EngenhariadeSoftware_para_CiênciadeDados/Churn
============================= test session starts ==============================
platform linux -- Python 3.11.13, pytest-8.4.1, pluggy-1.6.0
rootdir: /content/drive/MyDrive/ALURA/4812-EngenhariadeSoftware_para_CiênciadeDados/Churn
plugins: cov-6.2.1, anyio-4.9.0, langsmith-0.4.8, typeguard-4.4.4
collected 4 items                                                              

tests/test_core.py ....                                                  [100%]

================================ tests coverage ================================
_______________ coverage: platform linux, python 3.11.13-final-0 _______________

Name                                            Stmts   Miss  Cover
-------------------------------------------------------------------
src/tratamentos/pre_processamento/__init__.py       1      0   100%
src/tratamentos/pre_processamento/core.py          34      0   100%
----------------------